In [18]:
import pandas as pd
import numpy as np
import yfinance as yf


In [19]:
import yfinance as yf
import pandas as pd

# Download all historical data for AAPL
data = yf.download("AAPL", period="max")

# Keep only the Close column and rename it
df = data[["Close"]].rename(columns={"Close": "AAPL"}).reset_index()

# Ensure no multi-level columns remain
df.columns = ["Date", "AAPL"]

df.set_index('Date', inplace=True)



C:\Users\minhd\AppData\Local\Temp\ipykernel_27608\3596682501.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download("AAPL", period="max")
[*********************100%***********************]  1 of 1 completed


In [20]:
## long term mean level
def get_miu(df:pd.DataFrame)  -> float:
    return df.mean().iloc[0]

In [21]:
def get_Xt(df:pd.DataFrame)-> float:
    return df.iloc[-1,0]

In [22]:
def get_dt (df:pd.DataFrame) -> float:
    return 1

In [23]:
def get_theta(df:pd.DataFrame) -> float:
    ## time = 1, due to daily returns
    time = get_dt(df)
    rho = df.iloc[:,0].autocorr(lag=1)
    return -(1/time) * np.log(rho) 


In [24]:
def get_var(df:pd.DataFrame, theta:float, dt: float, mu:float) -> float:
    df = df.iloc[:,0].values
    exp1 = np.exp(-theta * dt)
    exp2 = np.exp(-2 * theta * dt)

    diff = df[1:] - df[:-1] * exp1 - mu * (1 - exp1)
    numerator = 2 * theta * np.sum(diff**2)
    denominator = len(df) * (1 - exp2)

    sigma2 = numerator / denominator
    return sigma2

In [25]:
def get_dXt(theta:float, mu: float, Xt:float, dt: float, var: float, dWt: float) -> float:
    return (theta * (mu - Xt)* (1/dt)) +( np.sqrt(var) * dWt)
 

In [26]:
mu = get_miu(df)
time = get_dt(df)
theta = get_theta(df)
X_t = get_Xt(df)
var = get_var(df, theta, time, mu)
dX_t = get_dXt(theta, mu, X_t, time, var, np.sqrt(time))

In [27]:
import sys
print(sys.executable, sys.version)  # sanity check: which Python is running?

# install yfinance into THIS exact interpreter
import subprocess, sys as _sys
subprocess.check_call([_sys.executable, "-m", "pip", "install", "--upgrade", "pip", "yfinance", "pandas"])


c:\Users\minhd\AppData\Local\Programs\Python\Python313\python.exe 3.13.3 (tags/v3.13.3:6280bb5, Apr  8 2025, 14:47:33) [MSC v.1943 64 bit (AMD64)]


0